In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

2017-08-26 23:23:13,692 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Model 1 evaluation

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-skip.model')
model.init_sims(replace=True)

2017-08-26 23:23:14,225 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model
2017-08-26 23:23:20,709 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model.wv.* with mmap=None
2017-08-26 23:23:20,710 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model.wv.syn0.npy with mmap=None
2017-08-26 23:23:29,540 : INFO : setting ignored attribute syn0norm to None
2017-08-26 23:23:30,835 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model.syn1neg.npy with mmap=None
2017-08-26 23:25:03,646 : INFO : setting ignored attribute cum_table to None
2017-08-26 23:25:04,089 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-skip.model
2017-08-26 23:26:57,571 : INFO : precomputing L2-norms of word weight vectors


In [3]:
num_features= 600

In [4]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [5]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [6]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [7]:
data_tokenized = df.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [8]:
%%time
data = word_averaging_list(model,data_tokenized)

2017-08-26 23:32:38,009 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:38,728 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:39,842 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:40,356 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:40,524 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:40,835 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:41,277 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:41,478 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:41,487 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:42,226 : WARNING : cannot compute similarity with no input []
2017-08-26 23:32:42,602 : WARNING : cannot compute similarity with no input []


CPU times: user 5.66 s, sys: 145 ms, total: 5.8 s
Wall time: 10.2 s


In [9]:
del model

## Random Forest

In [10]:
clf = RandomForestClassifier(max_depth=2, random_state=42, criterion= "gini", class_weight="balanced", n_estimators=1000)

## Extraversion

In [11]:
scores = cross_val_score(clf, data, df['extraversion_m'], cv=10, scoring='f1_macro')

In [12]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.60 (+/- 0.12)


## Agreableness

In [13]:
scores = cross_val_score(clf, data, df['agreeabeness_m'], cv=10, scoring='f1_macro')

In [14]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.54 (+/- 0.15)


## Consciousness

In [15]:
scores = cross_val_score(clf, data, df['conscientiousness_m'], cv=10, scoring='f1_macro')

In [16]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.57 (+/- 0.09)


## Neuroticism

In [17]:
scores = cross_val_score(clf, data, df['neuroticism_m'], cv=10, scoring='f1_macro')

In [18]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.54 (+/- 0.10)


## Openess

In [19]:
scores = cross_val_score(clf, data, df['openness_m'], cv=10, scoring='f1_macro')

In [20]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

F1-score: 0.58 (+/- 0.10)
